# Social Network Analysis applied to Data Warehouses

## 2. Network Connectivity Analysis: Out-Degree

The **degree** is simply the number of other nodes to which a node is adjacent (Scott, 2017). In directed networks, such as those used in this research, _out-degree_ refers to the number of connections that proceed from a particular node, and _in-degree_ is the number of connections it receives. A node’s degree is the sum of its in-degree and out-degree. The present analysis considered only the out-degree because it represents the downstream dependencies for a given data asset. From this perspective, assets with higher out-degrees are considered more important because their absence might cause data pipelines to break and downstream assets to become obsolete. 

### 2.1. Import dependencies

In [ ]:
import itertools
import math
from typing import Dict, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
from networkx.classes.graph import Graph
import numpy as np
import pandas as pd
from pandas import Series

### 2.2. Utility functions

In [ ]:
def load_graph_from_csv(file: str) -> Graph:
    return nx.read_edgelist(file, delimiter=",", create_using=nx.DiGraph)

In [ ]:
def format_graph_info(graph_id: str, graph: Graph) -> str:
    return (
        f"{graph_id.upper()} GRAPH INFO:\n"
        f"  Number of nodes: {nx.number_of_nodes(graph)}\n"
        f"  Number of edges: {nx.number_of_edges(graph)}\n"
        f"  Density: {nx.density(graph)}\n"
        f"  Average clustering coefficient: {nx.average_clustering(graph)}\n"
        f"  Transitivity: {nx.transitivity(graph)}"
    )

In [ ]:
def get_out_degree_list(graph: Graph) -> List[int]:
    return [degree for _, degree in graph.out_degree]

In [ ]:
def get_out_degree_series(graph: Graph) -> Series:
    return pd.Series(get_out_degree_list(graph))

In [ ]:
def draw_out_degree_based_network(graph: Graph) -> None:
    sorted_degrees = sorted(get_out_degree_list(graph))

    highest_degree = sorted_degrees[-1]
    scaling_factor = 3500 / highest_degree

    normalized_node_params = [
        (degree or 0.05) * scaling_factor for degree in sorted_degrees
    ]

    plt.figure(figsize=(12, 8))
    nx.draw_networkx(
        graph,
        pos=nx.spring_layout(graph),
        with_labels=False,
        edge_color="dimgray",
        node_color=normalized_node_params,
        node_size=normalized_node_params,
    )
    plt.axis("off")

In [ ]:
def plot_out_degree_descriptive_stats(graph: Graph, color: str) -> None:
    degrees = get_out_degree_list(graph)

    plt.figure(figsize=(12, 3))
    plt.boxplot(degrees, vert=False, flierprops=dict(markerfacecolor=color))
    plt.xlabel("Out-degree")

In [ ]:
def plot_out_degree_histogram(
    graph: Graph, color: str, start_from_degree: int = 0
) -> None:
    degrees = get_out_degree_list(graph)
    unique_degrees, counts = np.unique(degrees, return_counts=True)

    start_from_index = 0
    if start_from_degree:
        start_from_index = np.where(unique_degrees == start_from_degree)[0][0]

    plt.figure(figsize=(12, 3))
    plt.bar(unique_degrees[start_from_index:], counts[start_from_index:], color=color)
    plt.xlabel("Out-degree")
    plt.ylabel("# of Nodes")

In [ ]:
def plot_out_degree_ranking(graph: Graph, fmt: str, stop_at_degree: int = 0) -> None:
    degrees = sorted(get_out_degree_list(graph), reverse=True)

    if stop_at_degree:
        stop_at_index = degrees.index(stop_at_degree)
        while degrees[stop_at_index] == degrees[stop_at_index + 1]:
            stop_at_index += 1

    adjusted_list = [None]
    adjusted_list.extend(degrees[: stop_at_index + 1] if stop_at_degree else degrees)

    plt.figure(figsize=(12, 3))
    plt.plot(adjusted_list, fmt)
    plt.xlabel("Rank")
    plt.ylabel("Out-degree")

In [ ]:
def group_nodes_by_out_degree(
    graph: Graph, highest_first: bool = True
) -> Dict[int, List[str]]:
    degrees = get_out_degree_list(graph)
    unique_degrees = sorted(np.unique(degrees), reverse=highest_first)
    degrees_dict = {}
    for unique_degree in unique_degrees:
        degrees_dict[int(unique_degree)] = sorted(
            [node for node, degree in graph.out_degree if degree == unique_degree]
        )
    return degrees_dict

In [ ]:
def get_out_degree_critical_nodes_for_count(
    graph: Graph, target_node_count
) -> Tuple[Dict[int, List[str]], float]:
    node_count = 0
    degree_groups = group_nodes_by_out_degree(graph)
    degree_sum = sum(get_out_degree_list(graph))

    group_count = 0
    while node_count < target_node_count:
        group_count += 1
        highest_degrees = dict(itertools.islice(degree_groups.items(), group_count))
        node_count = sum([len(nodes) for _, nodes in highest_degrees.items()])

    highest_degree_sum = sum(
        [degree * len(nodes) for degree, nodes in highest_degrees.items()]
    )
    groups_degree_ratio = highest_degree_sum / degree_sum * 100
    print(
        f"{group_count}-critical-groups node count: {node_count}\n"
        f"{group_count}-critical-groups/total out-degree ratio:"
        f" {groups_degree_ratio:.0f}% ({highest_degree_sum}/{degree_sum})"
    )

    return highest_degrees, node_count

In [ ]:
def get_out_degree_critical_nodes_for_ratio(
    graph: Graph, target_ratio
) -> Tuple[Dict[int, List[str]], float]:
    groups_degree_ratio = 0
    degree_groups = group_nodes_by_out_degree(graph)
    degree_sum = sum(get_out_degree_list(graph))

    group_count = 0
    while groups_degree_ratio < target_ratio:
        group_count += 1
        highest_degrees = dict(itertools.islice(degree_groups.items(), group_count))
        highest_degree_sum = sum(
            [degree * len(nodes) for degree, nodes in highest_degrees.items()]
        )
        groups_degree_ratio = highest_degree_sum / degree_sum * 100

    node_count = sum([len(nodes) for _, nodes in highest_degrees.items()])

    print(
        f"{group_count}-critical-groups node count: {node_count}\n"
        f"{group_count}-critical-groups/total out-degree ratio:"
        f" {groups_degree_ratio:.0f}% ({highest_degree_sum}/{degree_sum})"
    )

    return highest_degrees, groups_degree_ratio

### 2.3. Load the anonymized graphs from CSV

In [ ]:
anon_data_folder = "../data/anonymized"

small_graph_1 = load_graph_from_csv(f"{anon_data_folder}/anon-dataset-small_1.csv")
print(f'{format_graph_info("small(1)", small_graph_1)}\n')

small_graph_2 = load_graph_from_csv(f"{anon_data_folder}/anon-dataset-small_2.csv")
print(f'{format_graph_info("small(2)", small_graph_2)}\n')

medium_graph = load_graph_from_csv(f"{anon_data_folder}/anon-dataset-medium.csv")
print(f'{format_graph_info("medium", medium_graph)}\n')

large_graph = load_graph_from_csv(f"{anon_data_folder}/anon-dataset-large.csv")
print(f'{format_graph_info("large", large_graph)}')

### 2.4. Draw the networks, degree histograms, and rankings

#### 2.4.1. Small network 1

In [ ]:
draw_out_degree_based_network(small_graph_1)

In [ ]:
sg_1_out_degree_series = get_out_degree_series(small_graph_1)
print(sg_1_out_degree_series.describe())

In [ ]:
plot_out_degree_descriptive_stats(small_graph_1, "red")

A common fact among the four datasets is that there are few nodes with high out-degrees, as summarized by the above descriptive statistics and box plot chart. It means there is no clear threshold that can be used to determine what are the most important assets from the out-degree perspective.

Still, out-degree is a discrete metric, so the nodes can be grouped by their out-degrees and then ranked accordingly. The assets belonging to groups with higher out-degrees are the most relevant. The next chart brings a histogram showing the groups corresponding to the lower out-degrees, with higher node counts, at left, and the groups with the most important assets from the center to the right.

In [ ]:
plot_out_degree_histogram(small_graph_1, "red")

In [ ]:
plot_out_degree_ranking(small_graph_1, "ro-")

The groups corresponding to ~20% of the total out-degrees were empirically selected to determine the most relevant assets. The results are presented in the next cells.

In [ ]:
sg_1_critical_groups, _ = get_out_degree_critical_nodes_for_ratio(small_graph_1, 20)
print(f"\n{sg_1_critical_groups}")

sg_1_less_critical_out_degree = list(sg_1_critical_groups)[-1]

In [ ]:
plot_out_degree_histogram(
    small_graph_1, "red", start_from_degree=sg_1_less_critical_out_degree
)

In [ ]:
plot_out_degree_ranking(
    small_graph_1, "ro-", stop_at_degree=sg_1_less_critical_out_degree
)

In [ ]:
# Only informative.

sg_1_critical_groups, _ = get_out_degree_critical_nodes_for_count(small_graph_1, 10)
print(f"\n{sg_1_critical_groups}")

#### 2.4.2. Small network 2

In [ ]:
draw_out_degree_based_network(small_graph_2)

In [ ]:
sg_2_out_degree_series = get_out_degree_series(small_graph_2)
print(sg_2_out_degree_series.describe())

In [ ]:
plot_out_degree_descriptive_stats(small_graph_2, "blue")

In [ ]:
plot_out_degree_histogram(small_graph_2, "blue")

In [ ]:
plot_out_degree_ranking(small_graph_2, "bo-")

In [ ]:
sg_2_critical_groups, _ = get_out_degree_critical_nodes_for_ratio(small_graph_2, 20)
print(f"\n{sg_2_critical_groups}")

sg_2_less_critical_out_degree = list(sg_2_critical_groups)[-1]

In [ ]:
plot_out_degree_histogram(
    small_graph_2, "blue", start_from_degree=sg_2_less_critical_out_degree
)

In [ ]:
plot_out_degree_ranking(
    small_graph_2, "bo-", stop_at_degree=sg_2_less_critical_out_degree
)

In [ ]:
# Only informative.

sg_2_critical_groups, _ = get_out_degree_critical_nodes_for_count(small_graph_2, 10)
print(f"\n{sg_2_critical_groups}")

#### 2.4.3. Medium network

In [ ]:
draw_out_degree_based_network(medium_graph)

In [ ]:
mg_out_degree_series = get_out_degree_series(medium_graph)
print(mg_out_degree_series.describe())

In [ ]:
plot_out_degree_descriptive_stats(medium_graph, "green")

In [ ]:
plot_out_degree_histogram(medium_graph, "green")

In [ ]:
plot_out_degree_ranking(medium_graph, "go-")

In [ ]:
mg_critical_groups, _ = get_out_degree_critical_nodes_for_ratio(medium_graph, 20)
print(f"\n{mg_critical_groups}")

mg_less_critical_out_degree = list(mg_critical_groups)[-1]

In [ ]:
plot_out_degree_histogram(
    medium_graph, "green", start_from_degree=mg_less_critical_out_degree
)

In [ ]:
plot_out_degree_ranking(medium_graph, "go-", stop_at_degree=mg_less_critical_out_degree)

A slightly different approach will be used to select the most important assets for the _anon-dataset-medium_ and _anon-dataset-large_ datasets: a target asset count is specified instead of a percentage. This is because they contain way more assets than the small datasets and the ~20% percentage might return hundreds of assets, which is not helpful for Data Governance teams to act. 

In [ ]:
mg_critical_groups, _ = get_out_degree_critical_nodes_for_count(medium_graph, 30)
print(f"\n{mg_critical_groups}")

mg_less_critical_out_degree = list(mg_critical_groups)[-1]

In [ ]:
plot_out_degree_histogram(
    medium_graph, "green", start_from_degree=mg_less_critical_out_degree
)

In [ ]:
plot_out_degree_ranking(medium_graph, "go-", stop_at_degree=mg_less_critical_out_degree)

#### 2.4.4. Large network

In [ ]:
draw_out_degree_based_network(large_graph)

In [ ]:
lg_out_degree_series = get_out_degree_series(large_graph)
print(lg_out_degree_series.describe())

In [ ]:
plot_out_degree_descriptive_stats(large_graph, "magenta")

In [ ]:
plot_out_degree_histogram(large_graph, "magenta")

In [ ]:
plot_out_degree_ranking(large_graph, "mo-")

In [ ]:
lg_critical_groups, _ = get_out_degree_critical_nodes_for_ratio(large_graph, 20)
print(f"\n{lg_critical_groups}")

lg_less_critical_out_degree = list(lg_critical_groups)[-1]

In [ ]:
plot_out_degree_histogram(
    large_graph, "magenta", start_from_degree=lg_less_critical_out_degree
)

In [ ]:
plot_out_degree_ranking(large_graph, "mo-", stop_at_degree=lg_less_critical_out_degree)

In [ ]:
lg_critical_groups, _ = get_out_degree_critical_nodes_for_count(large_graph, 30)
print(f"\n{lg_critical_groups}")

lg_less_critical_out_degree = list(lg_critical_groups)[-1]

In [ ]:
plot_out_degree_histogram(
    large_graph, "magenta", start_from_degree=lg_less_critical_out_degree
)

In [ ]:
plot_out_degree_ranking(large_graph, "mo-", stop_at_degree=lg_less_critical_out_degree)